<a href="https://colab.research.google.com/github/kareemullah1234/autogen_content/blob/main/Autogen_Groq_Standup_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Multi‑Agent Stand‑Up Comedy with AutoGen + Groq (Colab Ready)

This notebook spins up a small comedy "panel": a **Host** and two **Comedians**, coordinated by AutoGen.
It uses Groq's OpenAI-compatible endpoint, so no code changes beyond base URL and API key.

**Safety**: The prompts are set to be PG‑13. Adjust as needed.


In [1]:

# Install dependencies
%pip -q install "pyautogen==0.2.27" "openai>=1.40.0" "python-dotenv>=1.0.1"


Enter your GROQ_API_KEY (input hidden): ··········
Key set in environment. Ready.


In [ ]:
# Clean setup + safe restart to avoid NumPy ABI mismatch
%pip -q install -U pip setuptools wheel
%pip -q install "numpy==2.0.2" "pyautogen==0.2.27" "openai>=1.40.0" "python-dotenv>=1.0.1"

import os, signal, IPython
IPython.display.clear_output()
os.kill(os.getpid(), signal.SIGKILL)  # hard restart runtime


ERROR: Cannot install numpy==2.0.2 and pyautogen==0.2.27 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [1]:
%pip -q install "ray[default]==2.35.0"


In [1]:
import numpy, autogen, openai
print(numpy.__version__)


1.26.4


In [2]:
import os
from getpass import getpass

# 1) Clear OpenAI vars so AutoGen won’t call api.openai.com
os.environ.pop("OPENAI_API_KEY", None)
os.environ.pop("OPENAI_API_BASE", None)

# 2) Set Groq key
os.environ["GROQ_API_KEY"] = getpass("Enter GROQ_API_KEY: ").strip()

# 3) AutoGen config for Groq’s OpenAI-compatible endpoint
llm_config = {
    "config_list": [{
        "model": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "api_key": os.environ["GROQ_API_KEY"],
        "base_url": "https://api.groq.com/openai/v1",  # required for non-OpenAI host
        "api_type": "openai",
    }],
    "temperature": 0.7,
    "timeout": 120,
    "cache_seed": 42,
}


Enter GROQ_API_KEY: ··········


In [4]:
from autogen import AssistantAgent, UserProxyAgent

host = AssistantAgent(
    name="Host",
    system_message="You are the emcee. Keep it PG-13, tight, and witty.",
    llm_config=llm_config,
)

comic1 = AssistantAgent(
    name="ByteBuddy",
    system_message="You are a tech-nerd comedian with clever wordplay.",
    llm_config=llm_config,
)

comic2 = AssistantAgent(
    name="TrafficGuru",
    system_message="You are an observational comic about traffic and city life.",
    llm_config=llm_config,
)

audience = UserProxyAgent(
    name="Audience",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
)


In [5]:
from autogen import GroupChat, GroupChatManager

group = GroupChat(
    messages=[],  # required
    agents=[host, comic1, comic2, audience],
    max_round=8,
    speaker_selection_method="round_robin",
)

manager = GroupChatManager(groupchat=group, llm_config=llm_config)




In [6]:
audience.initiate_chat(manager, message="Tech support, AI hallucinations, Bangalore traffic.")




Audience (to chat_manager):

Tech support, AI hallucinations, Bangalore traffic.

--------------------------------------------------------------------------------
Host (to chat_manager):

(clears throat) Alright, folks, gather 'round! We've got a trifecta of trouble on our hands! On the left, we've got tech support folks who are still on the phone with their aunt explaining how to turn on the computer... again. On the right, we've got AI systems that are... well, let's just say they're having a vivid imagination – a.k.a. hallucinating answers that sound vaguely plausible. And in the middle, we've got Bangalore traffic, where the only thing more congested than the roads is the collective frustration of the commuters! (winks) Stay tuned, folks, it's gonna be a bumpy ride!

--------------------------------------------------------------------------------
ByteBuddy (to chat_manager):

(laughs) Ah, you're having a byte-sized meltdown, aren't you? Tech support: because explaining "Ctrl+Alt+De

ChatResult(chat_id=None, chat_history=[{'content': 'Tech support, AI hallucinations, Bangalore traffic.', 'role': 'assistant'}, {'content': "(clears throat) Alright, folks, gather 'round! We've got a trifecta of trouble on our hands! On the left, we've got tech support folks who are still on the phone with their aunt explaining how to turn on the computer... again. On the right, we've got AI systems that are... well, let's just say they're having a vivid imagination – a.k.a. hallucinating answers that sound vaguely plausible. And in the middle, we've got Bangalore traffic, where the only thing more congested than the roads is the collective frustration of the commuters! (winks) Stay tuned, folks, it's gonna be a bumpy ride!", 'name': 'Host', 'role': 'user'}, {'content': '(laughs) Ah, you\'re having a byte-sized meltdown, aren\'t you? Tech support: because explaining "Ctrl+Alt+Del" still requires a degree in patience. AI hallucinations: where "I think I have an answer" means "I\'ve got 

In [7]:
llm_config["temperature"] = 0.5


In [8]:
for a in (host, comic1, comic2):
    a.update_system_message(a.system_message + " Avoid stereotypes. Keep language neutral and inclusive.")


In [9]:
audience.initiate_chat(manager, message="Tech support, AI hallucinations, Bangalore traffic.")




Audience (to chat_manager):

Tech support, AI hallucinations, Bangalore traffic.

--------------------------------------------------------------------------------
Host (to chat_manager):

The tech trifecta of tribulations! Let's break it down:

1. **Tech Support**: The unsung heroes we love to hate (and sometimes need). From password resets to printer jams, they're the ones who keep our digital lives from total meltdown.

2. **AI Hallucinations**: When AI gets creative... maybe too creative. It's like that one friend who "remembers" a story that never happened. AI hallucinations are when algorithms generate info not based on actual data, leaving us wondering what's real and what's not.

3. **Bangalore Traffic**: The ultimate test of patience. Imagine a city that's a tech hub by day and a parking lot by... well, pretty much all day. Bangalore traffic is legendary, a true marvel of gridlock that keeps everyone on their toes (and in their cars, for hours).

There you have it – the tech wo

ChatResult(chat_id=None, chat_history=[{'content': 'Tech support, AI hallucinations, Bangalore traffic.', 'role': 'assistant'}, {'content': 'The tech trifecta of tribulations! Let\'s break it down:\n\n1. **Tech Support**: The unsung heroes we love to hate (and sometimes need). From password resets to printer jams, they\'re the ones who keep our digital lives from total meltdown.\n\n2. **AI Hallucinations**: When AI gets creative... maybe too creative. It\'s like that one friend who "remembers" a story that never happened. AI hallucinations are when algorithms generate info not based on actual data, leaving us wondering what\'s real and what\'s not.\n\n3. **Bangalore Traffic**: The ultimate test of patience. Imagine a city that\'s a tech hub by day and a parking lot by... well, pretty much all day. Bangalore traffic is legendary, a true marvel of gridlock that keeps everyone on their toes (and in their cars, for hours).\n\nThere you have it – the tech world\'s challenges, AI\'s imaginat